In [5]:
pip install openmeteo-requests


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: openmeteo-requests in c:\users\steph\appdata\local\programs\python\python312\lib\site-packages (1.3.0)




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install requests-cache


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install retry-requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import openmeteo_requests
import requests_cache
import retry_requests
from retry_requests import retry
import pandas as pd 
import numpy as np

In [17]:
colonnes=["date","pm10", "pm2_5", "carbon_monoxide", "carbon_dioxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone","latitude","longitude"]
df = pd.DataFrame(columns=colonnes)
lat=[12,15,23,65]
lon=[14,56,56,89]

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
	"latitude": lat,
	"longitude": lon,
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "carbon_dioxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone"],
	"timezone": "Europe/Berlin",
	"start_date": "2023-01-01",
	"end_date": "2023-12-31"
}
responses = openmeteo.weather_api(url, params=params)

for i in range(0,len(lat)):
# Process first location. Add a for-loop for multiple locations or weather models
	response = responses[i]
	print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
	print(f"Elevation {response.Elevation()} m asl")
	print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
	print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

	# Process hourly data. The order of variables needs to be the same as requested.
	hourly = response.Hourly()
	hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
	hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
	hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
	hourly_carbon_dioxide = hourly.Variables(3).ValuesAsNumpy()
	hourly_nitrogen_dioxide = hourly.Variables(4).ValuesAsNumpy()
	hourly_sulphur_dioxide = hourly.Variables(5).ValuesAsNumpy()
	hourly_ozone = hourly.Variables(6).ValuesAsNumpy()

	hourly_data = {"date": pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
		end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}
	hourly_data["pm10"] = hourly_pm10
	hourly_data["pm2_5"] = hourly_pm2_5
	hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
	hourly_data["carbon_dioxide"] = hourly_carbon_dioxide
	hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
	hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
	hourly_data["ozone"] = hourly_ozone
	hourly_data["longitude"] = np.full(len(hourly_pm10),params["longitude"][i])
	hourly_data["latitude"] = np.full(len(hourly_pm10),params["latitude"][i])

	hourly_dataframe = pd.DataFrame(data = hourly_data)
	df = pd.concat([df, hourly_dataframe], axis=0, ignore_index=True)


Coordinates 12.0°N 14.0°E
Elevation 293.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s
Coordinates 15.0°N 56.0°E
Elevation 0.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s
Coordinates 23.0°N 56.0°E
Elevation 196.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s
Coordinates 65.0°N 89.0°E
Elevation 411.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s


C:\Users\steph\AppData\Local\Temp\ipykernel_2212\1798151198.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, hourly_dataframe], axis=0, ignore_index=True)


In [14]:
df

,date,pm10,pm2_5,carbon_monoxide,carbon_dioxide,nitrogen_dioxide,sulphur_dioxide,ozone,latitude,longitude
0,2022-12-31 23:00:00+00:00,57.000000,20.100000,119.0,NaN,1.10,0.90,54.0,12,14
1,2023-01-01 00:00:00+00:00,56.000000,19.200001,119.0,NaN,1.10,0.95,53.0,12,14
2,2023-01-01 01:00:00+00:00,55.500000,18.400000,118.0,NaN,1.05,0.95,52.0,12,14
3,2023-01-01 02:00:00+00:00,53.200001,17.600000,117.0,NaN,1.00,1.00,51.0,12,14
4,2023-01-01 03:00:00+00:00,50.900002,17.200001,116.0,NaN,0.95,1.00,50.0,12,14
...,...,...,...,...,...,...,...,...,...,...
35035,2023-12-31 18:00:00+00:00,0.100000,0.000000,170.0,NaN,0.40,0.10,50.0,65,89
35036,2023-12-31 19:00:00+00:00,0.100000,0.000000,170.0,NaN,0.40,0.10,50.0,65,89
35037,2023-12-31 20:00:00+00:00,0.100000,0.100000,170.0,NaN,0.40,0.10,49.0,65,89
35038,2023-12-31 21:00:00+00:00,0.100000,0.100000,170.0,NaN,0.40,0.10,49.0,65,89


In [18]:
colonnes=["date","temperature_2m", "relative_humidity_2m", "precipitation", "rain", "snowfall", "snow_depth", "surface_pressure", "cloud_cover", "wind_speed_10m", "soil_temperature_0_to_7cm", "soil_moisture_0_to_7cm","latitude","longitude"]
df_temp = pd.DataFrame(columns=colonnes)
lat=[12,15,23,65]
lon=[14,56,56,89]

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": lat,
	"longitude": lon,
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "snowfall", "snow_depth", "surface_pressure", "cloud_cover", "wind_speed_10m", "soil_temperature_0_to_7cm", "soil_moisture_0_to_7cm"],
	"timezone": "Europe/Berlin",
	"start_date": "2023-01-01",
	"end_date": "2023-12-31"
}
responses = openmeteo.weather_api(url, params=params)


for i in range(0,len(lat)):
# Process first location. Add a for-loop for multiple locations or weather models
	response = responses[i]
	print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
	print(f"Elevation {response.Elevation()} m asl")
	print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
	print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

	# Process hourly data. The order of variables needs to be the same as requested.
	hourly = response.Hourly()
	hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
	hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
	hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
	hourly_rain = hourly.Variables(3).ValuesAsNumpy()
	hourly_snowfall = hourly.Variables(4).ValuesAsNumpy()
	hourly_snow_depth = hourly.Variables(5).ValuesAsNumpy()
	hourly_surface_pressure = hourly.Variables(6).ValuesAsNumpy()
	hourly_cloud_cover = hourly.Variables(7).ValuesAsNumpy()
	hourly_wind_speed_10m = hourly.Variables(8).ValuesAsNumpy()
	hourly_soil_temperature_0_to_7cm = hourly.Variables(9).ValuesAsNumpy()
	hourly_soil_moisture_0_to_7cm = hourly.Variables(10).ValuesAsNumpy()

	hourly_data = {"date": pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
		end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}
	hourly_data["temperature_2m"] = hourly_temperature_2m
	hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
	hourly_data["precipitation"] = hourly_precipitation
	hourly_data["rain"] = hourly_rain
	hourly_data["snowfall"] = hourly_snowfall
	hourly_data["snow_depth"] = hourly_snow_depth
	hourly_data["surface_pressure"] = hourly_surface_pressure
	hourly_data["cloud_cover"] = hourly_cloud_cover
	hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
	hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
	hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
	hourly_data["longitude"] = np.full(len(hourly_pm10),params["longitude"][i])
	hourly_data["latitude"] = np.full(len(hourly_pm10),params["latitude"][i])

	hourly_dataframe = pd.DataFrame(data = hourly_data)
	df_temp = pd.concat([df_temp, hourly_dataframe], axis=0, ignore_index=True)

Coordinates 11.985939979553223°N 13.97666072845459°E
Elevation 293.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s
Coordinates 15.008787155151367°N 55.96638870239258°E
Elevation 0.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s
Coordinates 23.022846221923828°N 55.956111907958984°E
Elevation 196.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s
Coordinates 64.9912109375°N 89.0°E
Elevation 411.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s


C:\Users\steph\AppData\Local\Temp\ipykernel_2212\1440400746.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_temp = pd.concat([df_temp, hourly_dataframe], axis=0, ignore_index=True)


In [19]:
df_temp = df_temp.drop(columns="date")
df=df.drop(columns=["longitude","latitude"])

In [20]:
df_final = pd.concat([df,df_temp], axis=1)

In [21]:
df_final.head(5)

,date,pm10,pm2_5,carbon_monoxide,carbon_dioxide,nitrogen_dioxide,sulphur_dioxide,ozone,temperature_2m,relative_humidity_2m,...,rain,snowfall,snow_depth,surface_pressure,cloud_cover,wind_speed_10m,soil_temperature_0_to_7cm,soil_moisture_0_to_7cm,latitude,longitude
0,2022-12-31 23:00:00+00:00,57.000000,20.100000,119.0,NaN,1.10,0.90,54.0,19.775999,26.405745,...,0.0,0.0,0.0,982.750305,15.0,14.007655,21.575998,0.028,12,14
1,2023-01-01 00:00:00+00:00,56.000000,19.200001,119.0,NaN,1.10,0.95,53.0,19.375999,26.874191,...,0.0,0.0,0.0,982.221558,8.0,14.168641,19.875999,0.028,12,14
2,2023-01-01 01:00:00+00:00,55.500000,18.400000,118.0,NaN,1.05,0.95,52.0,18.175999,28.238911,...,0.0,0.0,0.0,981.697449,0.0,12.722830,19.275999,0.028,12,14
3,2023-01-01 02:00:00+00:00,53.200001,17.600000,117.0,NaN,1.00,1.00,51.0,17.375999,29.055292,...,0.0,0.0,0.0,981.412048,0.0,12.397806,18.726000,0.028,12,14
4,2023-01-01 03:00:00+00:00,50.900002,17.200001,116.0,NaN,0.95,1.00,50.0,16.625999,29.482401,...,0.0,0.0,0.0,981.325012,0.0,11.183201,18.075998,0.028,12,14


In [25]:
df_final.shape

(35040, 24)

In [25]:
df_final["jour"] = df_final["date"].dt.date

In [29]:
df_final["lon-lat"] = df_final["longitude"].astype(str).str.cat(df_final["latitude"].astype(str),sep="_")

In [30]:
df_final.head()

,date,pm10,pm2_5,carbon_monoxide,carbon_dioxide,nitrogen_dioxide,sulphur_dioxide,ozone,temperature_2m,relative_humidity_2m,...,snow_depth,surface_pressure,cloud_cover,wind_speed_10m,soil_temperature_0_to_7cm,soil_moisture_0_to_7cm,latitude,longitude,jour,lon-lat
0,2022-12-31 23:00:00+00:00,57.000000,20.100000,119.0,NaN,1.10,0.90,54.0,19.775999,26.405745,...,0.0,982.750305,15.0,14.007655,21.575998,0.028,12,14,2022-12-31,14_12
1,2023-01-01 00:00:00+00:00,56.000000,19.200001,119.0,NaN,1.10,0.95,53.0,19.375999,26.874191,...,0.0,982.221558,8.0,14.168641,19.875999,0.028,12,14,2023-01-01,14_12
2,2023-01-01 01:00:00+00:00,55.500000,18.400000,118.0,NaN,1.05,0.95,52.0,18.175999,28.238911,...,0.0,981.697449,0.0,12.722830,19.275999,0.028,12,14,2023-01-01,14_12
3,2023-01-01 02:00:00+00:00,53.200001,17.600000,117.0,NaN,1.00,1.00,51.0,17.375999,29.055292,...,0.0,981.412048,0.0,12.397806,18.726000,0.028,12,14,2023-01-01,14_12
4,2023-01-01 03:00:00+00:00,50.900002,17.200001,116.0,NaN,0.95,1.00,50.0,16.625999,29.482401,...,0.0,981.325012,0.0,11.183201,18.075998,0.028,12,14,2023-01-01,14_12


In [ ]:
df_final_agg = df_final.groupby(["jour","lon-lat"]).agg{
    
}

In [22]:
df_final.corr()

,date,pm10,pm2_5,carbon_monoxide,carbon_dioxide,nitrogen_dioxide,sulphur_dioxide,ozone,temperature_2m,relative_humidity_2m,...,rain,snowfall,snow_depth,surface_pressure,cloud_cover,wind_speed_10m,soil_temperature_0_to_7cm,soil_moisture_0_to_7cm,latitude,longitude
date,1.000000e+00,0.208119,0.254539,0.184150,NaN,-0.000584,-0.061496,-0.180113,0.073502,0.099710,...,0.032945,-0.032648,-0.269589,-0.005415,0.000631,-0.029977,0.049246,0.050425,1.236841e-13,3.184820e-13
pm10,2.081195e-01,1.000000,0.858100,0.141791,NaN,0.139967,0.182137,0.103130,0.466057,-0.446297,...,-0.033376,-0.133957,-0.368387,0.077108,-0.174504,-0.095693,0.506066,-0.429974,-4.157948e-01,-4.087099e-01
pm2_5,2.545394e-01,0.858100,1.000000,0.329812,NaN,0.286204,0.163160,0.004996,0.505882,-0.338805,...,-0.030003,-0.170318,-0.404805,0.222492,-0.195243,-0.070615,0.485046,-0.500683,-5.318326e-01,-5.008116e-01
carbon_monoxide,1.841502e-01,0.141791,0.329812,1.000000,NaN,0.615675,0.146705,-0.008430,0.108564,-0.110500,...,0.039460,-0.036291,-0.260275,-0.189057,0.081091,-0.354176,0.102453,-0.000817,-1.766563e-01,-4.223570e-01
carbon_dioxide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nitrogen_dioxide,-5.839276e-04,0.139967,0.286204,0.615675,NaN,1.000000,0.469273,-0.092923,0.084615,-0.163781,...,-0.018073,-0.061167,-0.180320,-0.077363,-0.095373,-0.210751,0.077720,-0.153466,-1.651016e-01,-2.405253e-01
sulphur_dioxide,-6.149637e-02,0.182137,0.163160,0.146705,NaN,0.469273,1.000000,0.329634,0.013727,-0.180012,...,-0.040820,0.029609,0.019465,-0.029482,-0.143568,-0.034266,0.058481,-0.140826,5.102555e-03,9.773590e-02
ozone,-1.801134e-01,0.103130,0.004996,-0.008430,NaN,-0.092923,0.329634,1.000000,0.143950,-0.413196,...,-0.035346,-0.011868,0.093295,0.064106,-0.120019,0.008794,0.283425,-0.176046,-4.493404e-02,5.634122e-02
temperature_2m,7.350181e-02,0.466057,0.505882,0.108564,NaN,0.084615,0.013727,0.143950,1.000000,-0.526835,...,-0.002472,-0.254770,-0.780449,0.467299,-0.297656,-0.177859,0.942203,-0.798025,-8.212049e-01,-6.543782e-01
relative_humidity_2m,9.970959e-02,-0.446297,-0.338805,-0.110500,NaN,-0.163781,-0.180012,-0.413196,-0.526835,1.000000,...,0.083256,0.194463,0.481085,0.046913,0.229072,0.329528,-0.638836,0.490502,4.217166e-01,5.188089e-01
